In [1]:
import time
from simtk import openmm, unit
from simtk.openmm import app
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
import argparse 
import mdtraj as md
from tqdm import tqdm
import os
import pickle
import numpy as np
import copy
import logging

In [2]:
# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.DEBUG)


In [3]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

In [4]:
# Define simulation parameters
temperature = 300 * unit.kelvin
nsteps_eq = 1000 # 250 ns
nsteps_neq = 20000 # 80 ps
neq_splitting ='V R H O R V'
timestep = 2.0 * unit.femtosecond
platform_name = 'CUDA'

In [5]:
# Read in htf
with open(os.path.join("/data/chodera/zhangi/perses_benchmark/neq/8/8/", f"8_complex.pickle"), 'rb') as f:
    htf = pickle.load(f)
system = htf.hybrid_system
positions = htf.hybrid_positions


In [6]:
lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
lambda_protocol = LambdaProtocol(functions = 'default')
lambda_alchemical_state.set_alchemical_parameters(0.0, lambda_protocol)
thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature=temperature), composable_states=[lambda_alchemical_state])
zero_endstate = copy.deepcopy(thermodynamic_state)
one_endstate = copy.deepcopy(thermodynamic_state)
one_endstate.set_alchemical_parameters(1.0, lambda_protocol)


In [7]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=300 * unit.kelvin)



In [8]:
# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = one_endstate.create_context(integrator, platform=platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)



In [9]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



In [23]:
# Run equilibration
final_pos = np.empty(shape=(1001, htf.hybrid_topology.n_atoms, 3))
pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
final_pos[0] = pos * unit.nanometers
for step in range(nsteps_eq):
    initial_time = time.time()
    integrator.step(1)
    #     if step % 125000 == 0:
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    final_pos[step] = pos *unit.nanometers
    elapsed_time = (time.time() - initial_time) * unit.seconds
    _logger.info(f'Step: {step} took {elapsed_time} seconds')




INFO:root:Step: 0 took 0.008528709411621094 s seconds
INFO:root:Step: 1 took 0.009125947952270508 s seconds
INFO:root:Step: 2 took 0.006627559661865234 s seconds
INFO:root:Step: 3 took 0.006549358367919922 s seconds
INFO:root:Step: 4 took 0.0069577693939208984 s seconds
INFO:root:Step: 5 took 0.006495475769042969 s seconds
INFO:root:Step: 6 took 0.006452083587646484 s seconds
INFO:root:Step: 7 took 0.00682520866394043 s seconds
INFO:root:Step: 8 took 0.006490230560302734 s seconds
INFO:root:Step: 9 took 0.006879329681396484 s seconds
INFO:root:Step: 10 took 0.006791353225708008 s seconds
INFO:root:Step: 11 took 0.0065059661865234375 s seconds
INFO:root:Step: 12 took 0.007212400436401367 s seconds
INFO:root:Step: 13 took 0.0065975189208984375 s seconds
INFO:root:Step: 14 took 0.006561994552612305 s seconds
INFO:root:Step: 15 took 0.006943941116333008 s seconds
INFO:root:Step: 16 took 0.006569623947143555 s seconds
INFO:root:Step: 17 took 0.0072939395904541016 s seconds
INFO:root:Step: 1

INFO:root:Step: 149 took 0.006810665130615234 s seconds
INFO:root:Step: 150 took 0.006941080093383789 s seconds
INFO:root:Step: 151 took 0.00645899772644043 s seconds
INFO:root:Step: 152 took 0.007596254348754883 s seconds
INFO:root:Step: 153 took 0.006456613540649414 s seconds
INFO:root:Step: 154 took 0.006555080413818359 s seconds
INFO:root:Step: 155 took 0.007061958312988281 s seconds
INFO:root:Step: 156 took 0.0066983699798583984 s seconds
INFO:root:Step: 157 took 0.006498098373413086 s seconds
INFO:root:Step: 158 took 0.007093667984008789 s seconds
INFO:root:Step: 159 took 0.006643533706665039 s seconds
INFO:root:Step: 160 took 0.007519245147705078 s seconds
INFO:root:Step: 161 took 0.006594419479370117 s seconds
INFO:root:Step: 162 took 0.006562709808349609 s seconds
INFO:root:Step: 163 took 0.008902788162231445 s seconds
INFO:root:Step: 164 took 0.009124517440795898 s seconds
INFO:root:Step: 165 took 0.0077838897705078125 s seconds
INFO:root:Step: 166 took 0.007373332977294922 s

INFO:root:Step: 296 took 0.006453990936279297 s seconds
INFO:root:Step: 297 took 0.006430387496948242 s seconds
INFO:root:Step: 298 took 0.0068051815032958984 s seconds
INFO:root:Step: 299 took 0.0064697265625 s seconds
INFO:root:Step: 300 took 0.0068454742431640625 s seconds
INFO:root:Step: 301 took 0.006406068801879883 s seconds
INFO:root:Step: 302 took 0.007142305374145508 s seconds
INFO:root:Step: 303 took 0.006757974624633789 s seconds
INFO:root:Step: 304 took 0.006386995315551758 s seconds
INFO:root:Step: 305 took 0.006369590759277344 s seconds
INFO:root:Step: 306 took 0.006865262985229492 s seconds
INFO:root:Step: 307 took 0.006421089172363281 s seconds
INFO:root:Step: 308 took 0.00683140754699707 s seconds
INFO:root:Step: 309 took 0.006830453872680664 s seconds
INFO:root:Step: 310 took 0.006424665451049805 s seconds
INFO:root:Step: 311 took 0.006745576858520508 s seconds
INFO:root:Step: 312 took 0.006469249725341797 s seconds
INFO:root:Step: 313 took 0.00634002685546875 s secon

INFO:root:Step: 443 took 0.007138252258300781 s seconds
INFO:root:Step: 444 took 0.00740814208984375 s seconds
INFO:root:Step: 445 took 0.0072574615478515625 s seconds
INFO:root:Step: 446 took 0.009436845779418945 s seconds
INFO:root:Step: 447 took 0.00859212875366211 s seconds
INFO:root:Step: 448 took 0.007695198059082031 s seconds
INFO:root:Step: 449 took 0.007275819778442383 s seconds
INFO:root:Step: 450 took 0.006775617599487305 s seconds
INFO:root:Step: 451 took 0.0076372623443603516 s seconds
INFO:root:Step: 452 took 0.006796598434448242 s seconds
INFO:root:Step: 453 took 0.0066759586334228516 s seconds
INFO:root:Step: 454 took 0.0071353912353515625 s seconds
INFO:root:Step: 455 took 0.006730318069458008 s seconds
INFO:root:Step: 456 took 0.006685972213745117 s seconds
INFO:root:Step: 457 took 0.0071408748626708984 s seconds
INFO:root:Step: 458 took 0.006681680679321289 s seconds
INFO:root:Step: 459 took 0.007915258407592773 s seconds
INFO:root:Step: 460 took 0.00669407844543457 

INFO:root:Step: 590 took 0.006479740142822266 s seconds
INFO:root:Step: 591 took 0.006850242614746094 s seconds
INFO:root:Step: 592 took 0.006514072418212891 s seconds
INFO:root:Step: 593 took 0.006438016891479492 s seconds
INFO:root:Step: 594 took 0.0074007511138916016 s seconds
INFO:root:Step: 595 took 0.0064389705657958984 s seconds
INFO:root:Step: 596 took 0.006455183029174805 s seconds
INFO:root:Step: 597 took 0.006848335266113281 s seconds
INFO:root:Step: 598 took 0.006436347961425781 s seconds
INFO:root:Step: 599 took 0.006797313690185547 s seconds
INFO:root:Step: 600 took 0.006440401077270508 s seconds
INFO:root:Step: 601 took 0.006441831588745117 s seconds
INFO:root:Step: 602 took 0.007613420486450195 s seconds
INFO:root:Step: 603 took 0.006417512893676758 s seconds
INFO:root:Step: 604 took 0.0064051151275634766 s seconds
INFO:root:Step: 605 took 0.006849050521850586 s seconds
INFO:root:Step: 606 took 0.006472587585449219 s seconds
INFO:root:Step: 607 took 0.006811141967773437

INFO:root:Step: 736 took 0.006754159927368164 s seconds
INFO:root:Step: 737 took 0.007723093032836914 s seconds
INFO:root:Step: 738 took 0.006812095642089844 s seconds
INFO:root:Step: 739 took 0.007298469543457031 s seconds
INFO:root:Step: 740 took 0.00695347785949707 s seconds
INFO:root:Step: 741 took 0.006745576858520508 s seconds
INFO:root:Step: 742 took 0.007203817367553711 s seconds
INFO:root:Step: 743 took 0.00677800178527832 s seconds
INFO:root:Step: 744 took 0.007333993911743164 s seconds
INFO:root:Step: 745 took 0.006988525390625 s seconds
INFO:root:Step: 746 took 0.0063915252685546875 s seconds
INFO:root:Step: 747 took 0.006537914276123047 s seconds
INFO:root:Step: 748 took 0.006925344467163086 s seconds
INFO:root:Step: 749 took 0.006523847579956055 s seconds
INFO:root:Step: 750 took 0.00700688362121582 s seconds
INFO:root:Step: 751 took 0.006618499755859375 s seconds
INFO:root:Step: 752 took 0.007384061813354492 s seconds
INFO:root:Step: 753 took 0.007090091705322266 s secon

INFO:root:Step: 883 took 0.0063898563385009766 s seconds
INFO:root:Step: 884 took 0.006379127502441406 s seconds
INFO:root:Step: 885 took 0.0067784786224365234 s seconds
INFO:root:Step: 886 took 0.006482124328613281 s seconds
INFO:root:Step: 887 took 0.007112979888916016 s seconds
INFO:root:Step: 888 took 0.00677180290222168 s seconds
INFO:root:Step: 889 took 0.00774383544921875 s seconds
INFO:root:Step: 890 took 0.008579730987548828 s seconds
INFO:root:Step: 891 took 0.006997346878051758 s seconds
INFO:root:Step: 892 took 0.007634162902832031 s seconds
INFO:root:Step: 893 took 0.007187604904174805 s seconds
INFO:root:Step: 894 took 0.00648951530456543 s seconds
INFO:root:Step: 895 took 0.007218360900878906 s seconds
INFO:root:Step: 896 took 0.006940126419067383 s seconds
INFO:root:Step: 897 took 0.006391763687133789 s seconds
INFO:root:Step: 898 took 0.0068225860595703125 s seconds
INFO:root:Step: 899 took 0.006398439407348633 s seconds
INFO:root:Step: 900 took 0.006471395492553711 s 

In [38]:
from tqdm import tqdm

In [28]:
new_pos = np.empty(shape=(10, 32436, 3))
j = 0
for i in tqdm(range(0, 1000, 100)):
    new_pos[j] = htf.new_positions(final_pos[i] *unit.nanometer)
    j += 1

100%|██████████| 10/10 [00:09<00:00,  1.06it/s]


In [29]:
top_new = md.Topology.from_openmm(htf._topology_proposal.new_topology)
traj = md.Trajectory(new_pos, top_new)


In [34]:
traj.remove_solvent()

<mdtraj.Trajectory with 10 frames, 3146 atoms, 201 residues, without unitcells at 0x2b8f0a417048>

In [36]:
traj.save("traj_new.dcd")


In [37]:
traj[0].save("traj_new.pdb")